# Theory

#### Векторные представления слов (word embeddings)

Векторное представление слова (word embedding) — вещественный вектор в пространстве с фиксированной невысокой размерностью.

                                            Пример векторных представлений слов (2D t-SNE)
<img src='image/2D_tsne.PNG'>

Зачем нужны Word embeddings?
Сжатые векторные представления слов
1. полезны сами по себе, например, для поиска
синонимов или опечаток в поисковых запросах.
2. используются в качестве признаков для решения
самых различных задач: выявление именованных сущностей, тэгирование частей речи, машинный перевод, кластеризация документов, ранжирование документов, анализ тональности текста.

##### word2vec

Мера семантической близости — мера близости, предназначенная для количественной оценки семантической схожести слов. Такая мера показывает высокие значения для пар слов, находящихся в семантических отношениях (синонимия, ассоциативность и т.д.), и нулевые значения для всех остальных пар.

word2vec - алгоритм для получения векторных представлений слов. Подход основан на важной гипотезе, которую в науке принято называть гипотезой локальности — “слова, которые встречаются в одинаковых окружениях, имеют близкие значения”. Близость в данном случае понимается очень широко, как то, что рядом могут стоять только сочетающиеся слова. Например, для нас привычно словосочетание "заводной будильник". А сказать “заводной апельсин” мы не можем* — эти слова не сочетаются.

##### Алгоритм word2vec
Мы будем предсказывать вероятность слова по его окружению (контексту). То есть мы будем учить такие вектора слов, чтобы вероятность, присваиваемая моделью слову была близка к вероятности встретить это слово в этом окружении в реальном тексте.





<img src='image/w2v_formula.PNG'>

Здесь W0 — вектор целевого слова, Wc — это некоторый вектор контекста, вычисленный (например, путем усреднения) из векторов окружающих нужное слово других слов. А S — это функция, которая двум векторам сопоставляет одно число. Например, это может быть косинусное расстояние.

Процесс тренировки устроен следующим образом: мы берем последовательно (2k+1) слов, слово в центре является тем словом, которое должно быть предсказано. А окружающие слова являются контекстом длины по k с каждой стороны. Каждому слову в нашей модели сопоставлен уникальный вектор, который мы меняем в процессе обучения нашей модели. В целом, этот подход называется CBOW — continuous bag of words, continuous потому, что мы скармливаем нашей модели последовательно наборы слов из текста, a BoW потому что порядок слов в контексте не важен.
<img src='image/CBOW_.png'>
Другой подход skip-gram — прямо противоположный CBOW, то есть “словосочетание с пропуском”. Мы пытаемся из данного нам слова угадать его контекст (точнее вектор контекста). В остальном модель не претерпевает изменений.
<img src='image/skipgram.png'>

Что стоит отметить: хотя в модель не заложено явно никакой семантики, а только статистические свойства корпусов текстов, оказывается, что натренированная модель word2vec может улавливать некоторые семантические свойства слов. Классический пример:

<img src='image/word_embeddings.PNG'>

Слово "мужчина" относится к слову "женщина" так же, как слово "дядя" к слову "тётя", что для нас совершенно естественно и понятно, но в других моделям добиться такого же соотношения векторов можно только с помощью специальных ухищрений. Здесь же — это происходит естественно из самого корпуса текстов.

#### Что мы можем попробовать сделать с векторами слов?

Мы можем делать различные синтаксические, семантические NLP задачи с векторами слов, некоторое из них уже встроены. 


In [ ]:
import gensim.downloader as api

In [ ]:
api.info()['models'].keys()

dict_keys(['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis'])

In [ ]:
import gensim.downloader as api

word_vectors = api.load("glove-wiki-gigaword-100")  # загрузим предтренированные вектора слов из gensim-data
# выведим слово наиболее близкое к 'woman', 'king' и далекое от 'man'
result = word_vectors.most_similar(positive=['woman', 'king'], negative=['man'])
print("{}: {:.4f}".format(*result[0]))

In [ ]:
result = word_vectors.most_similar(positive=['women', 'king'], negative=['man'])
print("{}: {:.4f}".format(*result[0]))

KeyError: ignored

In [ ]:
# выведем лишнее слово
print(word_vectors.doesnt_match("breakfast cereal dinner lunch".split()))

print(word_vectors.doesnt_match("black green summer brown".split()))

ValueError: ignored

In [ ]:
# определим схожесть между словами
similarity = word_vectors.similarity('woman', 'man')
print(similarity)

similarity = word_vectors.similarity('human', 'man')
print(similarity)

similarity = word_vectors.similarity('bee', 'man')
print(similarity)

0.8323495
0.5288512
0.21199903


In [ ]:
# найдем top-3 самых близких слов
result = word_vectors.similar_by_word("man", topn=3)
print(result)

result = word_vectors.similar_by_word("cat", topn=3)
print(result)

result = word_vectors.similar_by_word("mouth", topn=3)
print(result)

[('woman', 0.832349419593811), ('boy', 0.7914870977401733), ('one', 0.7788748741149902)]
[('dog', 0.8798074722290039), ('rabbit', 0.7424426674842834), ('cats', 0.7323004007339478)]
[('tongue', 0.7366125583648682), ('mouths', 0.687748908996582), ('ear', 0.6811771988868713)]


## Simple chat-bot example

In [ ]:
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook

In [ ]:
!head -5 gazeta.csv

date,url,edition,topics,authors,title,text,reposts_fb,reposts_vk,reposts_ok,reposts_twi,reposts_lj,reposts_tg,likes,views,comm_count
2008-11-21 15:19:14,https://www.gazeta.ru/news/business/2008/11/21/n_1298950.shtml,-,Бизнес,,Госдума сокращает срок действия ставки экспортных пошлин на нефть,"Госдума приняла сегодня в первом чтении и сразу в целом поправки в закон «О таможенном тарифе», сокращающие срок действия ставки экспортных пошлин на нефть с 2-х до 1-го месяца.nnДля установления средних цен на нефть марки Urals и расчета экспортных пошлин правительство России в течение двух месяцев проводит мониторинг на международных рынках нефтяного сырья (средиземноморском и роттердамском), экспортные пошлины на нефть устанавливаются также раз в два месяца.nnСокращение на месяц периода мониторинга (с 15-го числа каждого календарного месяца по 14-е число следующего месяца) и соответственно срока действия ставок экспортных пошлин «позволит более оперативно реагировать на изменения экономической с

In [ ]:
counter_all = 0
counter_filter = 0
with open("prepared_answers.txt", "r") as f:
    for line in tqdm_notebook(f):
        counter_all +=1
        spls = line.split("\t")
        if len(spls[0].split()) < 2 or len(spls[1].split()) < 3 or len(spls[0].split()) > 15:
            continue
            
        counter_filter +=1

<ipython-input-3-73322b7b2d68>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(f):


In [ ]:
assert True

#Small preprocess of the answers

question = None
written = False

with open("prepared_answers.txt", "w") as fout:
    with open("Otvety.txt", "r") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

<ipython-input-6-4e0de9aa106b>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


KeyboardInterrupt: 

In [ ]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [ ]:
assert True

# Preprocess for models fitting

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with open("Otvety.txt", "r") as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 100000:
            break

<ipython-input-8-4964f3fb6446>:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


In [ ]:
sentences = [i for i in sentences if len(i) > 2]

In [ ]:
sentences[0]

['вопрос', 'тдв', 'отдыхать', 'лично', 'советовать', 'завести']

In [ ]:
Word2Vec?

In [ ]:
modelW2V = Word2Vec(sentences=sentences, vector_size=300, window=5, min_count=1)

In [ ]:
modelFT = FastText(sentences=sentences, vector_size=300, min_count=1, window=5, workers=8)

In [ ]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0

with open("prepared_answers.txt", "r") as f:
    for line in tqdm_notebook(f):
        n_w2v = 0
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        
        vector_w2v = np.zeros(300)
        vector_ft = np.zeros(300)
        for word in question:
            if word in modelW2V.wv:
                vector_w2v += modelW2V.wv[word]
                n_w2v += 1
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)
            
        counter += 1
        
        if counter > 100000:
            break

w2v_index.build(10)
ft_index.build(10)

<ipython-input-16-64550af31b99>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(f):


True

In [ ]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 3, )
    return [index_map[i] for i in answers]

In [ ]:
TEXT = "какой город самы красивый"

In [ ]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['<br> <br>. \n',
 'жить в родных краях, кататься по всему миру. \n',
 'Конечно. Это город в Дагестане на узком проходе между Каспийским морем и предгорьями Кавказа. Дербент — самый южный город Российской Федерации. <br>Дербент — иногда считают одним из древнейших «живущих» городов мира, старейшим городом Российской Федерации. Первые поселения возникли здесь в эпоху ранней бронзы — в конце 4 тысячелетия до н. э. . Первое упоминание Каспийских ворот — наиболее древнего названия Дербента — относится к VI в. до н. э. , его приводит известный древнегреческий географ Гекатей Милетский.. \n']

In [ ]:
get_response(TEXT, ft_index, modelFT, index_map)

['г Санкт-Петербург.. \n',
 'Шеньчжень <br> <br> <br> <br>Город расположен в 120 км от Гуанчжоу. Одна из первых Специальных Экономических Зон Китая. Шэньчжэнь возник на месте бывшей рыбацкой деревушки на границе с Гонконгом. За 20 лет превратился в современный город с населением в 3 млн. человек. Въезд китайцев в Шэньчжэнь ограничен. В городе самый высокий уровень жизни в Китае. В городе и пригородах находятся предприятия Гонконга, Тайваня, Сингапура, Японии, Южной Кореи, США, стран Западной Европы. Свыше 80% производимой продукции экспортируется. <br> <br>http://travel.gala.net/ref/cn/_enchzhen/ <br>Многие иностранные посетители говорят, что Шэньчжэнь - это то, что было бы с Китаем, если бы не коммунизм. Шэнчжэнь является горячей туристической точкой в Китае. Здесь возможно прекрасно провести отпуск: пляжи, тематические парки, отличные отели. Но настоящая жизнь в Шеньчжене начинается ночью. Культурной жизнью города заправляют частные компании, чем объясняется необыкновенное разнообраз

# HW
Урок 3. Embedding word2vec fasttext
Задача поиск похожих по эмбеддингам
Скачиваем датасет

!wget https://github.com/ods-ai-ml4sg/proj_news_viz/releases/download/data/gazeta.csv.gz

# пример работы с ним 
from corus import load_ods_gazeta
path = 'gazeta.csv.gz'
records = load_ods_gazeta(path)
next(records)

что надо сделать
1.   на основе word2vec/fasttext реализовать метод поиска ближайших статей (на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших статей к этому запросу)
2.   Проверить насколько хорошо работают подходы



## Imports

In [1]:
!pip install pymorphy2 -q
!pip install stop_words -q
!pip install annoy -q

     |████████████████████████████████| 55 kB 4.1 MB/s 
     |████████████████████████████████| 8.2 MB 48.3 MB/s 
     |████████████████████████████████| 646 kB 16.9 MB/s 


In [26]:
import string


import annoy
import gensim
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
import tqdm
from tqdm import tqdm_notebook

In [3]:
import pandas as pd
import numpy as np
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words

#from stop_words import get_stop_words
#import annoy
from gensim.models import Word2Vec, FastText
#import pickle
from tqdm import tqdm_notebook

## Data downloading

In [4]:
from os.path import exists

if not(exists('gazeta.csv.gz')):
  !wget https://github.com/ods-ai-ml4sg/proj_news_viz/releases/download/data/gazeta.csv.gz -q
if not(exists('gazeta.csv')):
  !7z x gazeta.csv.gz
  if exists('gazeta.csv.gz'):
    !rm 'gazeta.csv.gz'


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 477029050 bytes (455 MiB)

Extracting archive: gazeta.csv.gz
--
Path = gazeta.csv.gz
Type = gzip
Headers Size = 21

  0% - gazeta.csv                   1% - gazeta.csv                   2% - gazeta.csv                   3% - gazeta.csv                   4% - gazeta.csv                   5% - gazeta.csv                   6% - gazeta.csv                   7% - gazeta.csv                   8% - gazeta.csv                 

## Data processing

### Papers processing

In [5]:
def preprocess_txt(line):
    spls = line.split(',')[3:6]
    try:
      spls.remove('')
    except ValueError:
      pass
    finally:
      spls = ' '.join(spls)
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [6]:
def text_preprocessing(text):
  try:
    updated_text = text.lower()
  except :
    updated_text = text
  return updated_text

In [7]:
# memory consumption
"""papers = pd.read_csv('gazeta.csv')
print(papers.columns)

drop_columns_list = ['url','edition', 'text', 'reposts_fb', 'reposts_vk', 
                     'reposts_ok', 'reposts_twi', 'reposts_lj',
                     'reposts_tg', 'likes', 'views', 'comm_count']
papers.drop(columns=drop_columns_list, inplace=True)

# older news in the end of dataframe
papers.sort_values('date', ascending=False, inplace=True)

for column in ['topics', 'authors', 'title']:
  papers[column] = papers[column].apply(text_preprocessing)"""

"papers = pd.read_csv('gazeta.csv')\nprint(papers.columns)\n\ndrop_columns_list = ['url','edition', 'text', 'reposts_fb', 'reposts_vk', \n                     'reposts_ok', 'reposts_twi', 'reposts_lj',\n                     'reposts_tg', 'likes', 'views', 'comm_count']\npapers.drop(columns=drop_columns_list, inplace=True)\n\n# older news in the end of dataframe\npapers.sort_values('date', ascending=False, inplace=True)\n\nfor column in ['topics', 'authors', 'title']:\n  papers[column] = papers[column].apply(text_preprocessing)"

In [8]:
sentences = []
c = 0
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
file_length = 20000

with open("gazeta.csv", "r") as fin:
    for line in tqdm_notebook(fin):
      spls = preprocess_txt(line)
      sentences.append(spls)
      c += 1
      if c > file_length:
          break

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


0it [00:00, ?it/s]

## Model preparation/training

### Russian corpus
'день_NOUN' - words in vocab - problem

In [ ]:
if not(exists('180.zip')):
  !wget http://vectors.nlpl.eu/repository/20/180.zip -q

In [ ]:
import zipfile
model_file = '180.zip'
with zipfile.ZipFile(model_file, 'r') as archive:
   stream = archive.open('model.bin')
   model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

In [ ]:
words = ['день_NOUN', 'ночь_NOUN', 'человек_NOUN', 'семантика_NOUN', 'студент_NOUN', 'понедельник_NOUN']
for word in words:
  if word in model:
    print(word)
    similarity = model.similarity('дата_NOUN', word)
  else:
    similarity = 0
  print(similarity)

день_NOUN
0.24648403
ночь_NOUN
-0.058418483
человек_NOUN
-0.116438575
семантика_NOUN
0.13351995
студент_NOUN
-0.0667172
понедельник_NOUN
0.20837255


In [ ]:
'человек_NOUN' in  model.vocab

True

### Word2Vec

In [9]:
modelW2V = Word2Vec(min_count=2, sentences=sentences, window=5)
modelW2V.save("word2vec.model")

### FastText

In [10]:
modelFT = FastText(sentences=sentences, min_count=2, window=5)
modelFT.save("fasttext.model")

## Word2Vec and FastText recommendations

In [11]:
papers_number = 5
input_request = 'новости экономика'

In [32]:
counter = 0
lines2request_similarity_w2v = pd.DataFrame(columns=['line', 'similarity'])
lines2request_similarity_ft = pd.DataFrame(columns=['line', 'similarity'])

with open("gazeta.csv", "r") as f:
  for line in tqdm.notebook.tqdm(f):
    if counter > 0:
      corrected_line = preprocess_txt(line)
      counter += 1
      line2request_similarity_w2v = 0
      line2request_similarity_ft = 0
      for word_news in corrected_line:
        for word_request in preprocess_txt(input_request):
          if (word_request in modelW2V)and(word_news in modelW2V):
            line2request_similarity_w2v += modelW2V.wv.similarity(word_news, word_request)
          if (word_request in modelFT)and(word_news in modelFT):
            line2request_similarity_ft += modelFT.wv.similarity(word_news, word_request)
      lines2request_similarity_w2v = lines2request_similarity_w2v.append({'line':line.split(',')[0],
                                                                          'similarity':line2request_similarity_w2v},
                                                                         ignore_index=True)
      lines2request_similarity_ft = lines2request_similarity_ft.append({'line':line.split(',')[0],
                                                                        'similarity':line2request_similarity_ft},
                                                                       ignore_index=True)
      if not(counter%(2*papers_number)):
        lines2request_similarity_w2v = lines2request_similarity_w2v.sort_values(by=['similarity'],
                                                                                ignore_index=True,
                                                                                na_position='first')[-5:]
        lines2request_similarity_ft = lines2request_similarity_ft.sort_values(by=['similarity'], 
                                                                              ignore_index=True,
                                                                              na_position='first')[-5:] 
      if counter > 2*file_length:
        
        break
    else:
      counter += 1

lines2request_similarity_w2v = lines2request_similarity_w2v.sort_values(by=['similarity'],
                                                                        ignore_index=True,
                                                                        na_position='first')[-5:]
lines2request_similarity_ft = lines2request_similarity_ft.sort_values(by=['similarity'], 
                                                                      ignore_index=True,
                                                                      na_position='first')[-5:]
print(f'Word2Vec recommendations (file line with score):\n{lines2request_similarity_w2v}')
print(f'FastText recommendations (file line with score):\n{lines2request_similarity_ft}')

0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  app.launch_new_instance()


Word2Vec recommendations (file line with score):
                  line  similarity
1  2018-03-27 06:59:56  701.822227
2  2018-02-11 15:48:05  706.990806
3  2014-07-17 19:21:42  714.852607
4  2013-03-13 19:31:00  716.593466
5  2018-09-14 08:36:27  771.362024
FastText recommendations (file line with score):
                  line  similarity
1  2014-07-17 19:21:42  719.348175
2  2018-03-27 06:59:56  855.942173
3  2018-12-31 08:22:24  887.223314
4  2018-02-11 15:48:05  895.066923
5  2018-03-26 08:43:41  918.171655


In [33]:
with open("gazeta.csv", "r") as f:
  for line in tqdm_notebook(f):
    if line.split(',')[0] in list(lines2request_similarity_w2v['line']):
      print(f'Word2Vec recommendation:\n{line}\n')
    if line.split(',')[0] in list(lines2request_similarity_ft['line']):
      print(f'FastText recommendation:\n{line}\n')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


0it [00:00, ?it/s]

Word2Vec recommendation:
2013-03-13 19:31:00,https://www.gazeta.ru/social/2013/03/12/5054161.shtml,-,Общество,"Анастасия Берсенева,Максим Солопов,Дарья Загвоздина",Онлайн-репортаж из Ватикана: выборы папы римского,"Новый глава Римско-католической церкви избран с третьего раза: вечером в среду из трубы Сикстинской капеллы, в которой собрался конклав из 115 кардиналов Римско-католической церкви, показался белый дым. 266-м понтификом стал кардинал из Аргентины Хорхе Марио Бергольо.nnВпервые за всю историю Римско-католической церкви престол Святого Петра занял выходец из Латинской Америки, принадлежащий к ордену иезуитов. Папа римский Франциск I обратился к верующим, собравшимся на площади, с небольшой речью. Онвозблагодарил Бога за то, что католики всего мира получили нового понтифика, выразил благодарность кардиналам, избравшим его, а также собравшимся на площади перед собором святого Петра в Риме. конце своего выступления Франциск попросил собравшихся попросить у Бога, чтобы Он вдохнови